#### Student Name:
#### Student ID:

# CSE 190 Assignment 2

### Hidden Markov Model, LZify, and Speech Formants with Linear Predictive Coding

Instructions: 

* This notebook is an interactive assignment; please read and follow the instructions in each cell. 

* Cells that require your input (in the form of code or written response) will have 'Question #' above.

* After completing the assignment, please submit this notebook as a PDF and a WAV file of your Fur Elise variation.

## HMM-Based Chord Recognition

In music, certain chord transitions are more likely than others. This idea can be applied as Hidden Markov Models, where the first-order temporal relationships between the various chords are captured by the transition probability matrix $A$. 

If we consider only major and minor chords, there are a total of 24 chords in this model (12 major, from C through B, and 12 minor, from C through B), formalized as:

\begin{equation}
\label{eq:ChordReco:HMM:App:Spec:SetStates}
  \mathcal{A} = \{\mathbf{C},\mathbf{C}^\sharp,\ldots,\mathbf{B},\mathbf{Cm},\mathbf{Cm^\sharp},\ldots,\mathbf{Bm}\} 
\end{equation}

We use the notation $\alpha{i}\rightarrow\alpha{j}$ referring to the transition from state $\alpha{i}$ to state $\alpha{j}$, $i,j\in[1:24]$. For example, the coefficient $a_{1,2}$ expresses the 
probability for the transition $\alpha_{1}\rightarrow\alpha_{2}$ (corresponding to  $\mathbf{C}\rightarrow\mathbf{C}^\sharp$), whereas $a_{1,8}$ expresses the probability for $\alpha_{1}\rightarrow\alpha_{8}$  (corresponding to  $\mathbf{C}\rightarrow\mathbf{G}$). In real music, the change from a tonic to the dominant is much more likely
than transposing by one semitone, so that the probability $a_{1,8}$ should be much larger than $a_{1,2}$. The coefficients $a_{i,i}$ express the probability of staying in state $\alpha_{i}$ (i.e., $\alpha_{i}\rightarrow\alpha_{i}$), $i\in[1:24]$. These coefficients are also referred to as **self-transition** probabilities.

A transition probability matrix can be specified in many ways. For example, the matrix may be defined manually by a music expert based on rules from harmony theory. The most common approach is to generate such a matrix automatically 
by estimating the transition probabilities from labeled data. 

In the following exercise, you will train an HMM on transition probabilities found in the music of the Beatles using bigrams (pairs of adjacent elements) in labeled frame sequences from a subset of the Beatles Corpus.

For this assignment, assume each row in the dataset represents a chord that has followed the chord on the previous row in a Beatles song. When parsing the file for your model, you may discard any chord references beyond key and major/minor quality. For example, if a row reads 'Bbm7', you would parse for the key (B-flat) and quality (minor), but discard the extra information that it is a 7th chord.  

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from numpy.random import multinomial as randm
from numpy import where
import scipy.signal as si
import IPython.display as ipd
import librosa
import scipy
from matplotlib import patches
import librosa.display as ld

np.random.seed(42)

data = pd.read_csv('data/Liverpool_band_chord_sequence.csv')

##### Question 1

Using the above data, generate a 24x24 matrix, where each matrix element (i,j) is the transition probability from chord i to chord j. 

In [7]:
HMM = np.zeros((24,24))

### Your code here: 




In [ ]:
def plot_transition_matrix(A, ax=None, xlabel='State $a_j$', ylabel='State $a_i$', title='', clim=[-6, 0], cmap='gray_r'):
    
    im = ax[0].imshow(A, origin='lower', aspect='auto', cmap=cmap)
    im.set_clim([-6, 0])
    plt.sca(ax[0])
    cbar = plt.colorbar(im)
    ax[0].set_xlabel(xlabel)
    ax[0].set_ylabel(ylabel)
    ax[0].set_title(title)
    cbar.ax.set_ylabel('Probability')
    
    chroma_label = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chord_label_maj = chroma_label
    chord_label_min = [s + 'm' for s in chroma_label]
    chord_labels = chord_label_maj + chord_label_min
    chord_labels_squeezed = chord_labels.copy()
    for k in [13, 15, 17, 18, 20, 22]:
        chord_labels_squeezed[k] = ''
        
    ax[0].set_xticks(np.arange(24))
    ax[0].set_yticks(np.arange(24))
    ax[0].set_xticklabels(chord_labels_squeezed)
    ax[0].set_yticklabels(chord_labels)
    
    return im

plot_transition_matrix(HMM)

##### Question 2

Using your HMM, you will create your own 16-measure Beatles hits!
For your first song, beginning on C major, select each next chord by choosing the chord with the largest transition probability from the current chord:


In [3]:
my_first_beatles_hit = ''

### Your code here:


print(my_first_beatles_hit)

##### Question 3

For your second song, beginning on C major, select each next chord at random according to the probabilities of your HMM.
For example, if C major transitions to G major with probability .5, F major with probability .25, and D minor with probability .25, then your next chord should be selected randomly from (G, F, Dm) with probability of selection (.5, .25, .25) respectively. 

In [5]:
my_second_beatles_hit = ''

### Your code here:


print(my_second_beatles_hit)

## LZify: Applying Universal Prediction to Musical Style

In this section, you will implement the Incremental Parsing (IP) from the LZ78 method for creating a dictionary of motifs. These motifs are later used to generate new sequences resembling the input sequence.

Please read the algorithm in Assayag, Dubnov, and Delerue's "Guessing the Composer’s Mind" (available at https://pdfs.semanticscholar.org/0181/236e1b417c8dd5dddd1f919583893f7a9026.pdf). 

The IPMotif function should compute the motif dictionary discovered in the text. It uses Incremental Parsing method to parse the text into unseen motifs.

##### Question 4

In [ ]:
def IPMotif(text):
    """Compute an associative dictionary (the motif dictionary)."""
    
    dictionary = {}
    
    ### Your Code Here:

    
    
    return dictionary


The text below contains an excerpt of Beethoven's Fur Elise, written as MML (Music Macro Language). MML is used to represent musical melodies as text. 

You can read more about MML syntax here: https://en.wikipedia.org/wiki/Music_Macro_Language

You can play with MML in this webapp: http://benjaminsoule.fr/tools/vmml/

Try playing the text below in the webapp to hear sample output. 

In [ ]:
text = 'o6ed+ed+ec-dc>aceabeg+bb+e<ed+ed+ec-dc>aceabe<c>bab<cde>g<fed>f<edc>e<dc>be<eeed+ed+ec-dc>aceabeg+bb+e<ed+ed+ec-dc>aceabe<c>ba'
dict1 = IPMotif(text)
print(dict1)

Next, implement the IPContinuation and Normalize functions. 
The IPContinuation function transforms the IPMotif dictionary into a tree-like representation to allow finding continuations for new  motifs. The Normalize function turns the counters in every element of the IPContinuation dictionary into probabilities. 

##### Question 5

In [ ]:
def IPContinuation(dict1):
    """Compute continuation dictionary from a motif dictionary"""
    
    dict2 = {}

    ### Your Code Here: 
    
    
    return dict2

##### Question 6

In [ ]:
def Normalize(dict2):
    """Turns the counters in every element of dict2 to probabilities"""

    ### Your Code Here:
    

    return dict2

In [ ]:
dict2 = IPContinuation(dict1)
print(dict2)

Generting a new sequence is done by traversing the IPContinuation tree and selecting possible branches according to their weights. If motif is not found, its last symbol is removed and the process is repeated for a shorter motif.

In [ ]:
def IPGenerate(n,dict2):
    p = 0
    out = ""
    for k in range(n):
        while True:
            context = out[-p:]
            if context in dict2:
                prob = [tup[1] for tup in dict2[context]]
                conti = where(randm(1,prob))[0][0]
                cont = dict2[context][conti][0]
                out = out + cont
                break
            else:
                p = p-1
    return out
out = IPGenerate(92,dict2)
print(out)

##### Question 7

Paste your output in the online MML player, and listen to your piece. Do you hear elements of Fur Elise in your composition? What are some of the differences in the output from the original? 

Please export the WAV file of your output from the webapp, and submit the WAV file with your assignment. 

``` Your response here ```

A few important points:
1. The method captures the "texture" of the language but not it's meaning.
2. We could parse a new text using IPMotifs from two languages, then count the length and number of motifs in order to decide what was the language of the new text.
3. In order to use this method with musical information, we need first to translate audio to features, or in case of polyphonic midi change this into some proper representation. One possibility is using virtual fundamental or chroma for harmony, or some other specialized representation to capture repetition in terms of other specific musical properties.

## Speech Formants and LPC

In this section, you will synthesize vowel sounds, and investigate the frequencies in vowels from your own voice. 

In [ ]:
Fdict = {
    'mystery_1':[[328, 2208, 2885],[27,80,575]],
    'mystery_2':[[504, 868, 2654],[62,   108,  299]],
    'mystery_3':[[700, 1220, 2600],[130,   70,  160]]
    } # Formant frequencies in Hz

def excitation(f0,jitt,dur,nharm=None,unvoiced=False):
    w0T = 2*np.pi*f0/fs

    if nharm == None:
        nharm = int((fs/2)/f0) # number of harmonics
    nsamps = fs*dur
    sig = np.zeros(nsamps)
    ph = np.random.uniform(size=nsamps)*2*np.pi
    n = np.arange(nsamps)

    if unvoiced:
        sig = np.random.normal(size=nsamps)
    else:
    # Synthesize bandlimited impulse train
        for i in range(1,nharm):
            sig = sig + np.cos(i*w0T*n + jitt*ph)
    
    sig = sig/max(sig)
    return sig

def voca(sig,F,Fb):
    R = np.exp(-np.pi*Fb/fs);     # Pole radii
    theta = 2*np.pi*F/fs;     # Pole angles
    poles = R * np.exp(1j*theta) # Poles[B,A] = zpk2tf(0,np.concatenate((poles, np.conj(poles))),1) # Filter from zeros and poles
    
    [B,A] = si.zpk2tf(0,np.concatenate((poles, np.conj(poles))),1) # Filter from zeros and poles

    speech = si.lfilter(B, A, sig)
    speech = speech/np.std(speech)
    return speech,B,A

def J(x):
    x = (x - np.mean(x))/np.std(x)
    kurt = np.mean(np.power(x,4)) - 3*np.power(np.mean(np.power(x,2)),2)
    return 1./12*np.power(np.mean(np.power(x,3)),2) + 1./48*kurt

fs = 8192 # 22050  % Sampling rate in Hz ("telephone quality" for speed)

vowels = list(Fdict.keys())
f0 = 150 # Pitch in Hz
dur = 1 #one second in duration
ji = 0.1 #0.1
ex = excitation(f0,ji,dur)

text = ['mystery_1','mystery_2','mystery_3']

speech = np.zeros(1)
for t in text:
    F = np.array(Fdict[t][0])
    Fb = np.array(Fdict[t][1])
    print(t)

    vow,B,A = voca(ex,F,Fb)
    speech = np.concatenate((speech,vow))

speech = speech/np.std(speech)
plt.plot(speech)

In [ ]:
ipd.Audio(speech, rate=fs) 

##### Question 8

Based on the audio output, what vowels were synthesized as mystery_1, mystery_2, and mystery_3? 
Please specify using a word; for example, if you heard an 'oo' sound as in 'hoot', you may answer with the word "hoot". 

mystery_1:

mystery_2:

mystery_3:

Now we will examine just one vowel in greater detail. 
Select one mystery vowel to analyze below: 

In [ ]:
%matplotlib inline  

### Modify the line below:
text = ['SELECT_ONE_MSYERY_VOWEL']

speech = np.zeros(1)
for t in text:
    F = np.array(Fdict[t][0])
    Fb = np.array(Fdict[t][1])
    print(t)

    vow,B,A = voca(ex,F,Fb)
    speech = np.concatenate((speech,vow))

speech = speech/np.std(speech)
plt.plot(speech)

In [ ]:
plt.psd(speech, 1024)
plt.show()

In [ ]:
lpc_order = 10
s = speech

a = librosa.core.lpc(s, lpc_order)
print(a)
s_hat = scipy.signal.lfilter([0] + -1*a[1:], [1], s)
s_err = s[1:] - s_hat[:-1]
plt.plot(s[1:])
plt.plot(s_hat[:-1], linestyle='--')
plt.legend(['y', 'y_hat'])
plt.title('LP Model Forward Prediction')
plt.show()

In [ ]:
plt.plot(s[1:101])
plt.plot(s_hat[:100])
plt.legend(['y', 'y_hat'])
plt.show()

##### Question 9

What is being visualized as y and y_hat on the above plot?

``` Your response here ```

In [ ]:
w,h = si.freqz(b=1,a = a, fs=1)
plt.plot(w,20*np.log10(h))

In [ ]:
z,p,k = si.tf2zpk(B,A)
    
unit_circle = patches.Circle((0,0), radius=1, fill=False, color='black', ls='solid', alpha=0.9)
ax = plt.subplot(111)
ax.add_patch(unit_circle)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
r = 1.5; plt.axis('scaled'); plt.axis([-r, r, -r, r])
ticks = [-1, -.5, .5, 1]; plt.xticks(ticks); plt.yticks(ticks)    
    
plt.plot(z.real, z.imag, 'ko', fillstyle='none', ms = 10)
plt.plot(p.real, p.imag, 'kx', fillstyle='none', ms = 10)

In [ ]:
D = np.abs(librosa.stft(s,n_fft=256,hop_length = 64))
ld.specshow(librosa.amplitude_to_db(D))

##### Question 10

Record yourself speaking the same vowel sound you analyzed above. 
Graph the power spectral density of your recording alongside the LPC spectrum generated above. 

In [10]:
### Your Code Here




##### Question 11

How does the power spectral density of your recorded signal compare to the LPC spectrum? 

``` Your response here ```